In [1]:
import os,sys
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PeriodicTable
from rdkit.Chem import rdPartialCharges
from rdkit.Chem import Descriptors
import pandas as pd

df = pd.read_excel('generated_molecules_layers8_units[16, 16].xlsx') ####LOCATION OF THE EXCEL FILE
pse = Chem.GetPeriodicTable()
au_to_debye = 2.5412
orca_executable_location = '/Applications/orca/orca'  ###JUST INCLUDE THE LOCATION OF THE ORCA EXECUTABLE

In [2]:
def calculate_spin_multiplicity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        raise ValueError("Invalid SMILES string")

    unpaired_electrons = Descriptors.NumRadicalElectrons(mol)
    spin_multiplicity = 2 * (unpaired_electrons / 2) + 1

    return int(spin_multiplicity)

def get_total_charge(mol):
    # Parse the SMILES string into a molecule object

    # Calculate the total charge
    total_charge = sum([atom.GetFormalCharge() for atom in mol.GetAtoms()])
    return total_charge

def smiles_to_atoms(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        raise ValueError("Invalid SMILES string")

    m2 = Chem.AddHs(mol)
    AllChem.EmbedMolecule(m2, maxAttempts=5000)
    AllChem.UFFOptimizeMolecule(m2, maxIters=5000)
    
    rdPartialCharges.ComputeGasteigerCharges(m2)
    contribs = [x.GetDoubleProp('_GasteigerCharge') for x in m2.GetAtoms()]
    charge = sum(contribs)
    charge = round(charge)

    charge = round(get_total_charge(m2))
    atoms = []
    positions = []
    
    for atom in m2.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        pos = m2.GetConformer().GetAtomPosition(atom.GetIdx())
        atoms.append(atomic_num)
        positions.append((pos.x, pos.y, pos.z))

    return atoms, positions, charge 


In [3]:
og_dir = os.getcwd()


In [4]:
def thermo(home_dir, smiles_strings):
    dft_method = 'UHF 3-21G'   ### CAN YOU TRY RUNNING XTB 
    calc_type = ' Opt Freq'
    first_line = '! ' + str(dft_method) + calc_type + '\n'
    input_type = 'xyz'
    property_strings = ['INNERENERGYU', 'ENTHALPY', 'ENTROPY', 'FREEENERGYG', 'DIPOLEMAGNITUDE', 'ZPE']
    dipole = np.zeros(len(smiles_strings), dtype='float32')
    zpe = np.zeros(len(smiles_strings), dtype='float32')
    int_energy = np.zeros(len(smiles_strings), dtype='float32')
    enthalpy= np.zeros(len(smiles_strings), dtype='float32')
    entropy= np.zeros(len(smiles_strings), dtype='float32')
    free_energy= np.zeros(len(smiles_strings), dtype='float32')
    
    for i, smile in enumerate(smiles_strings):
        temp_smile = smile.split()[0]
        atomic_nos, atom_pos, charge = smiles_to_atoms(temp_smile)
        spin_mult = calculate_spin_multiplicity(temp_smile)
        second_line = '*' + str(input_type) + ' ' + str(charge) + ' ' + str(spin_mult) + '\n'
        input_file_string = str(i)
        new_dir = home_dir + '/' + input_file_string
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
        os.chdir(new_dir)
        if os.path.exists(input_file_string + '.xyz'):
            opt_file = open(input_file_string + '.xyz', 'r')
            xyz_lines = opt_file.readlines()
            atom_pos = []
            for j in range(2, len(xyz_lines)):
                splits = xyz_lines[j].split()
                atom_pos.append([float(splits[1]), float(splits[2]), float(splits[3])])
            
        orca_input = open(input_file_string + '.inp', 'w')
        orca_input.write(first_line)
        orca_input.write(second_line)
        for j in range(len(atomic_nos)):
            orca_input.write(str(pse.GetElementSymbol(atomic_nos[j]) + '\t'))
            orca_input.write(str(atom_pos[j][0]) + '\t' + str(atom_pos[j][1]) + '\t' + str(atom_pos[j][2]) + '\t')
            orca_input.write('\n')
        orca_input.write('*')
        orca_input.close()
        os.system(orca_executable_location + ' ' + input_file_string + '.inp > output.log 2>&1')
        if os.path.exists(input_file_string + '.property.txt'):

            prop_files = open(input_file_string + '.property.txt','r')
            prop_lines = prop_files.readlines()

        if os.path.exists(input_file_string + '_property.txt'):

            prop_files = open(input_file_string + '_property.txt','r')
            prop_lines = prop_files.readlines()

        for line in prop_lines:
            if property_strings[0] in line:
                splits = line.split()
                int_energy[i] = float(splits[-1])
            if property_strings[1] in line:
                splits = line.split()
                enthalpy[i] = float(splits[-1])
            if property_strings[2] in line:
                splits = line.split()
                entropy[i] = float(splits[-1])
            if property_strings[3] in line:
                splits = line.split()
                free_energy[i] = float(splits[-1])       
            if property_strings[4] in line:
                splits = line.split()
                dipole[i] = float(splits[-1])       
            if property_strings[5] in line:
                splits = line.split()
                zpe[i] = float(splits[-1])       

    file = open(home_dir + '/thermodynamic_properties','w')
    file.write('SMILES \t INTERNAL ENERGY \t ENTHALPY \t ENTROPY \t FREE ENERGY \t DIPOLE MAGNITUDE \t ZPE \n') 
    file.close()
    for i in range(len(smiles_strings)):
        file = open(home_dir + '/thermodynamic_properties','a')
        file.write(smiles_strings[i].rstrip() + '\t' +  str(int_energy[i]) + '\t' +  str(enthalpy[i]) + '\t' +  str(entropy[i]) + '\t' +  str(free_energy[i]) + '\t' +  str(dipole[i]) + '\t' + str(zpe[i]) + '\n')
        file.close()


In [5]:
home_dir = og_dir + '/' + 'B3LYP_SMILES_1' 
smiles_strings = df['Generated_SMILES_1']
thermo(home_dir, smiles_strings)
os.chdir(og_dir)

In [6]:
home_dir = og_dir + '/' + 'B3LYP_SMILES_2' 
smiles_strings = df['Generated_SMILES_2']
thermo(home_dir, smiles_strings) 
os.chdir(og_dir)

In [7]:
home_dir = og_dir + '/' + 'B3LYP_SMILES_3' 
smiles_strings = df['Generated_SMILES_3']
thermo(home_dir, smiles_strings) 
os.chdir(og_dir)

[02:25:13] UFFTYPER: Warning: hybridization set to SP3 for atom 2
[02:25:14] UFFTYPER: Warning: hybridization set to SP3 for atom 2


ValueError: Bad Conformer Id

In [6]:
home_dir = og_dir + '/' + 'B3LYP_SMILES_4' 
smiles_strings = df['Generated_SMILES_4']
thermo(home_dir, smiles_strings) 
os.chdir(og_dir)

In [7]:
home_dir = og_dir + '/' + 'B3LYP_SMILES_5' 
smiles_strings = df['Generated_SMILES_5']
thermo(home_dir, smiles_strings) 
os.chdir(og_dir)